In [ ]:
import pandas as pd
import numpy as np
import requests 
import datetime as dt
import json
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.dates
import csv
import glob
import random
import os
import sklearn
from sklearn import linear_model

### Data Analysis Code
#### Note: this code does NOT need to be run in order to run a simulation. This can be used to view data anaysis and model creation.
#### Note: You must download data yourself from sources cited in section headings.

### ASHP Data Analysis Code
#### Note: data to run this code is NOT provided. Published data can be downloaded from Catapult Energy Systems: https://es.catapult.org.uk/report/electrification-of-heat-interim-heat-pump-performance-data-analysis-report/

##### Load Data from Files:

In [ ]:
def ASHP_GetPropertyData(property_id,file_properties_base):
    file_property = "".join([file_properties_base,str(property_id),'.csv'])
    try:
        return pd.read_csv(file_property)
    except:
        print(f"Failed to open file: {property_id}, at location: {file_property}.")

# ASHP data: Electrification of Heat Demonstration Project:
# https://es.catapult.org.uk/project/electrification-of-heat-demonstration/
# for properties included in the analysis with ASHP type:
def ASHP_LoadPropertyIdsFromCsv(folder_location, filename):
    file_property_ids = folder_location+filename
    try:
        file_pointer = open(file_property_ids, "r")
    except:
        print(f"Failed to open file: {file_property_ids}")
    else:
        try: 
            i = 0
            property_ids = []
            for file_line in csv.reader(file_pointer):
                property_id = file_line[0]
                i+=1
                # skip header:
                if property_id[0] != "E":
                    continue
                # # select every nth property to analyse:
                # if i % 4 != 0:
                #     continue
                property_ids.append(property_id)
            return property_ids
        except: 
            print(f"Failed to open file: {property_id}.")

    
def ASHP_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data):
    property_ids = ASHP_LoadPropertyIdsFromCsv(folder_location,append_location_property_ids)
    property_data_unprocessed_all = []
    for property_id in property_ids:
        property_data_unprocessed =  ASHP_GetPropertyData(property_id,folder_location+folder_location_property_data)
        property_data_unprocessed_all.append(property_data_unprocessed)
    return (property_ids,property_data_unprocessed_all)


folder_location = ""
append_location_property_ids = 'PropertyIds.csv'
folder_location_property_data = '9050csv_cleansed_data_set1_b693745c14a63a7ed1c6299c5abe1a19/Property_ID='
(ashp_property_ids,ashp_property_data_unprocessed_all) = ASHP_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data)

##### Filter and Sort Data for Analysis:

In [ ]:
def ASHP_ProcessData(property_data_unprocessed):
    # split data by day to find daily averages:
    # data is recorded in 2 minute intervals:
    interval_mins = 2
    # how many entries are in one day:
    n_day = int((24*60)/interval_mins)
    # index values by day:
    cumulative_pump_energy_output = property_data_unprocessed["Heat_Pump_Energy_Output"][::n_day]
    cumulative_total_energy_consumed = property_data_unprocessed["Whole_System_Energy_Consumed"][::n_day]
    cumulative_immersion_energy_consumed = property_data_unprocessed["Immersion_Heater_Energy_Consumed"][::n_day]
    cumulative_circ_pump_energy_consumed = property_data_unprocessed["Circulation_Pump_Energy_Consumed"][::n_day]
    # convert non-cumulative data to cumulative sums and index:
    cumulative_external_temperatures = np.cumsum(property_data_unprocessed["External_Air_Temperature"])[::n_day]
    cumulative_internal_temperatures = np.cumsum(property_data_unprocessed["Internal_Air_Temperature"])[::n_day]
    # calculate daily totals for cumulative sums:
    daily_total_pump_energy_output = np.diff(cumulative_pump_energy_output)
    daily_total_energy_consumed = np.diff(cumulative_total_energy_consumed)
    daily_total_immersion_energy_consumed = np.diff(cumulative_immersion_energy_consumed)
    daily_total_circ_pump_energy_consumed = np.diff(cumulative_circ_pump_energy_consumed)
    daily_total_external_temperatures = np.diff(cumulative_external_temperatures)
    daily_total_internal_temperatures = np.diff(cumulative_internal_temperatures)
    # calculate daily averages for temperature sums:
    daily_average_pump_energy_output = daily_total_pump_energy_output
    daily_average_total_energy_consumed = daily_total_energy_consumed
    daily_average_immersion_energy_consumed = daily_total_immersion_energy_consumed
    daily_average_circ_pump_energy_consumed = daily_total_circ_pump_energy_consumed
    daily_average_external_temperature = daily_total_external_temperatures/n_day
    daily_average_internal_temperature = daily_total_internal_temperatures/n_day
    
    # calculate difference in internal vs external temp:
    # daily_average_temperature_diff = daily_average_internal_temperature - daily_average_external_temperature
    daily_average_temperature_diff = daily_average_external_temperature - daily_average_internal_temperature
    # calculate the total energy output by heating system:
    daily_average_heating_system_energy_output = daily_average_pump_energy_output + daily_average_immersion_energy_consumed
    
    # calculate the total energy used by heating system (H3):
    daily_average_total_energy_consumed_H3 = daily_average_total_energy_consumed - daily_average_circ_pump_energy_consumed
    
    # calculate the total energy used by heating system (H4):
    daily_average_total_energy_consumed_H4 = daily_average_total_energy_consumed
   
    # calculate daily average COPs:
    # COP (H3):
    daily_cop_H3 = daily_average_heating_system_energy_output/daily_average_total_energy_consumed_H3
    # COP (H4):
    daily_cop_H4 = daily_average_heating_system_energy_output/daily_average_total_energy_consumed_H4

    # timestamps for start of each day (date of daily average):
    # need to remove last entry as no data for this day:
    timestamp_start_of_day = [pd.to_datetime(ts) for ts in (property_data_unprocessed["Timestamp"][::n_day])]
    timestamp_start_of_day.pop()

    # add data to new dataframe:
    property_data = pd.DataFrame()
    property_data["start_of_day"] = timestamp_start_of_day
    # property_data["daily_average_pump_energy_output"] = daily_average_pump_energy_output
    property_data["daily_average_heating_energy_output"] = daily_average_heating_system_energy_output
    property_data["daily_average_total_energy_consumed_H3"] = daily_average_total_energy_consumed_H3
    property_data["daily_average_total_energy_consumed_H4"] = daily_average_total_energy_consumed_H4
    property_data["daily_average_external_temperature"] = daily_average_external_temperature
    property_data["daily_average_internal_temperature"] = daily_average_internal_temperature
    property_data["daily_average_temperature_diff"] = daily_average_temperature_diff
    property_data["daily_cop_H3"] = daily_cop_H3
    property_data["daily_cop_H4"] = daily_cop_H4

    # return processed data:
    return property_data

def ASHP_ProcessPropertiesData(property_ids,property_data_unprocessed_all):
    property_ids_accepted = []
    property_data_all = []
    polynomial_regression_model_all = []
    i=0
    for property_id in property_ids:
        property_data_unprocessed = property_data_unprocessed_all[i]
        i+=1
        property_data = ASHP_ProcessData(property_data_unprocessed)
        if(property_data is None):
            print(f"Property {property_id} rejected from analysis")
            property_ids_accepted.append(property_id)
            property_data_all.append(None)
        else:
            property_ids_accepted.append(property_id)
            property_data_all.append(property_data)

    return (property_ids_accepted,property_data_all)

(ashp_property_ids_accepted,ashp_property_data_all) = ASHP_ProcessPropertiesData(ashp_property_ids,ashp_property_data_unprocessed_all)

##### Data Analysis and Calculation of Metrics:

In [ ]:
def ASHP_ProcessPropertiesData(property_ids,property_data_unprocessed_all):
    property_ids_accepted = []
    property_data_all = []
    polynomial_regression_model_all = []
    i=0
    for property_id in property_ids:
        property_data_unprocessed = property_data_unprocessed_all[i]
        i+=1
        property_data = ASHP_ProcessData(property_data_unprocessed)
        if(property_data is None):
            print(f"Property {property_id} rejected from analysis")
            property_ids_accepted.append(property_id)
            property_data_all.append(None)
        else:
            property_ids_accepted.append(property_id)
            property_data_all.append(property_data)

    return (property_ids_accepted,property_data_all)

def CalculateAverageCop(property_data_unfiltered):
    # filter data for heat energy output >= 20kWh
    property_data = property_data_unfiltered[property_data_unfiltered["daily_average_heating_energy_output"] >= 20]
    # filter data for daily_average_temperature_diff <= -5
    property_data = property_data[property_data["daily_average_temperature_diff"] <= -5]
    # filter data for cop > 1.0
    property_data = property_data[property_data["daily_cop_H3"] >= 1]
    # calculate flat average COP for selected data range:
    return np.mean(property_data["daily_cop_H3"])

def CalculateAverageCop2(property_data_unfiltered):
    # fit linear model for energy out vs energy used:
    property_data = property_data_unfiltered.dropna()
    x = property_data["daily_average_total_energy_consumed_H3"].values
    y = property_data["daily_average_heating_energy_output"].values
    # print(x)
    # print(y)
    coeffs = np.polyfit(x,y, deg=1)
    # print(coeffs)
    return coeffs[0]

def ASHP_CopSummaryData(property_ids,property_data_all):
    # for temp diff > 7 degrees C, there appears to be correlation.
    # assuming beyond that point the heat pump is not operating fully / at capacity.
    # for each property, calculate the average COP in that range (flat average) to determine the "overall COP value"
    # to use for categorisation of heat pumps by COP.
    property_cop_summary_data_all = pd.DataFrame()
    average_cops = []
    average_cops_rounded = []
    i=0
    for property_id in property_ids:
        property_data = property_data_all[i]
        i+=1
        # cop data:
        average_cop = CalculateAverageCop2(property_data)
        average_cops.append(average_cop)
        # # round to nearest integer:
        # average_cops_rounded.append(round(average_cop,ndigits=0))
        # round to nearest 0.5:
        average_cops_rounded.append(round(2*average_cop,ndigits=0)/2)

    property_cop_summary_data_all["property_id"] = property_ids
    property_cop_summary_data_all["average_cop"] = average_cops
    property_cop_summary_data_all["average_cop_rounded"] = average_cops_rounded
    return property_cop_summary_data_all

def FitPolyModel(temp_data_all,cop_data_all,degree):
    # remove any rows with NaN values:
    model_coeffs = np.polyfit(temp_data_all, cop_data_all, deg=degree)
    poly_model = np.poly1d(model_coeffs)
    return (model_coeffs,poly_model)

def ASHP_LinearRegressionCopVsTempDiff(property_ids,property_cop_summary_data_all,property_data_all):
    # distinct COPs:
    cop_range = list(float(i) for i in set(property_cop_summary_data_all["average_cop_rounded"]))
    cop_range.sort()
    # polynomial regression model for each cop bound:
    poly_models = pd.DataFrame()
    cop_band_data_all = []
    cop_band_data_all_filt = []
    fitted_models = []
    fitted_models_filt = []
    for cop in cop_range:
        # ids = property_cop_summary_data_all[property_cop_summary_data_all["average_cop_rounded"]==cop]["property_id"]
        idx = np.array(property_cop_summary_data_all["average_cop_rounded"]==cop)
        property_data_cfilt = pd.Series(property_data_all)[idx]
        # combine properties data for temp diff and cop:
        temp_data_all = np.array([])
        cop_data_all = np.array([])

        temp_data_all_filt = np.array([])
        cop_data_all_filt = np.array([])

        cop_band_data = pd.DataFrame()
        cop_band_data_filt = pd.DataFrame()
        for property_data in property_data_cfilt:
            # drop NaNs:
            property_data = property_data.dropna()
            # filter data for heat energy output >= 20kWh
            property_data_tfilt = property_data[property_data["daily_average_heating_energy_output"] >= 20]
            # filter data for daily_average_temperature_diff <= 0
            property_data_tfilt = property_data_tfilt[property_data_tfilt["daily_average_temperature_diff"] <= -5]
            # filter data for cop > 1.0
            property_data_tfilt = property_data_tfilt[property_data_tfilt["daily_cop_H3"] >= 1]

            temp_data_filt = np.array(property_data_tfilt["daily_average_temperature_diff"].values)
            cop_data_filt = np.array(property_data_tfilt["daily_cop_H3"].values)

            temp_data = np.array(property_data["daily_average_temperature_diff"].values)
            cop_data = np.array(property_data["daily_cop_H3"].values)

            temp_data_all = np.concatenate((temp_data_all,temp_data))
            cop_data_all = np.concatenate((cop_data_all,cop_data))

            temp_data_all_filt = np.concatenate((temp_data_all_filt,temp_data_filt))
            cop_data_all_filt = np.concatenate((cop_data_all_filt,cop_data_filt))
        
        cop_band_data["temp_data_all"] = temp_data_all
        cop_band_data["cop_data_all"] = cop_data_all
        cop_band_data_all.append(cop_band_data)

        cop_band_data_filt["temp_data_all"] = temp_data_all_filt
        cop_band_data_filt["cop_data_all"] = cop_data_all_filt
        cop_band_data_all_filt.append(cop_band_data_filt)

        # fitted_model = FitPolyModel(temp_data_all,cop_data_all,1)
        fitted_model_filt = FitPolyModel(temp_data_all_filt,cop_data_all_filt,1)
        # fitted_models.append(fitted_model)
        fitted_models_filt.append(fitted_model_filt)

    poly_models["COP"] = cop_range
    # poly_models["fitted_models"] = fitted_models
    poly_models["fitted_models_filt"] = fitted_models_filt
    poly_models["cop_band_data_all"] = cop_band_data_all
    poly_models["cop_band_data_all_filt"] = cop_band_data_all_filt
    return (poly_models)

def ASHP_LinearRegressionCopVsTempDiff2(property_ids,property_cop_summary_data_all,property_data_all):
    # multivariate linear regression:
    
    # distinct COPs:
    cop_range = list(float(i) for i in set(property_cop_summary_data_all["average_cop_rounded"]))
    cop_range.sort()
    # polynomial regression model for each cop bound:
    poly_models = pd.DataFrame()
    cop_band_data_all = []
    cop_band_data_all_filt = []
    fitted_models = []
    fitted_models_filt = []
    for cop in cop_range:
        # ids = property_cop_summary_data_all[property_cop_summary_data_all["average_cop_rounded"]==cop]["property_id"]
        idx = np.array(property_cop_summary_data_all["average_cop_rounded"]==cop)
        property_data_cfilt = pd.Series(property_data_all)[idx]
        # combine properties data for temp diff and cop:
        temp_data_all = np.array([])
        cop_data_all = np.array([])

        temp_data_all_filt = np.array([])
        cop_data_all_filt = np.array([])

        cop_band_data = pd.DataFrame()
        cop_band_data_filt = pd.DataFrame()
        for property_data in property_data_cfilt:
            # drop NaNs:
            property_data = property_data.dropna()
            # filter data for heat energy output >= 20kWh
            property_data_tfilt = property_data[property_data["daily_average_heating_energy_output"] >= 20]
            # filter data for daily_average_temperature_diff <= 0
            property_data_tfilt = property_data_tfilt[property_data_tfilt["daily_average_temperature_diff"] <= -5]
            # filter data for cop > 1.0
            property_data_tfilt = property_data_tfilt[property_data_tfilt["daily_cop_H3"] >= 1]

            temp_data_filt = np.array(property_data_tfilt["daily_average_temperature_diff"].values)
            cop_data_filt = np.array(property_data_tfilt["daily_cop_H3"].values)

            temp_data = np.array(property_data["daily_average_temperature_diff"].values)
            cop_data = np.array(property_data["daily_cop_H3"].values)

            temp_data_all = np.concatenate((temp_data_all,temp_data))
            cop_data_all = np.concatenate((cop_data_all,cop_data))

            temp_data_all_filt = np.concatenate((temp_data_all_filt,temp_data_filt))
            cop_data_all_filt = np.concatenate((cop_data_all_filt,cop_data_filt))
        
        cop_band_data["temp_data_all"] = temp_data_all
        cop_band_data["cop_data_all"] = cop_data_all
        cop_band_data_all.append(cop_band_data)

        cop_band_data_filt["temp_data_all"] = temp_data_all_filt
        cop_band_data_filt["cop_data_all"] = cop_data_all_filt
        cop_band_data_all_filt.append(cop_band_data_filt)

        # fitted_model = FitPolyModel(temp_data_all,cop_data_all,1)
        fitted_model_filt = FitPolyModel(temp_data_all_filt,cop_data_all_filt,1)
        # fitted_models.append(fitted_model)
        fitted_models_filt.append(fitted_model_filt)

    poly_models["COP"] = cop_range
    # poly_models["fitted_models"] = fitted_models
    poly_models["fitted_models_filt"] = fitted_models_filt
    poly_models["cop_band_data_all"] = cop_band_data_all
    poly_models["cop_band_data_all_filt"] = cop_band_data_all_filt
    return (poly_models)

def ASHP_LinearRegressionEnergyVsTempDiff(property_ids,property_cop_summary_data_all,property_data_all):
    # distinct COPs:
    cop_range = list(float(i) for i in set(property_cop_summary_data_all["average_cop_rounded"]))
    cop_range.sort()
    # polynomial regression model for each cop bound:
    poly_models = pd.DataFrame()
    cop_band_data_all = []
    cop_band_data_all_filt = []
    fitted_models = []
    fitted_models_filt = []
    for cop in cop_range:
        # ids = property_cop_summary_data_all[property_cop_summary_data_all["average_cop_rounded"]==cop]["property_id"]
        idx = np.array(property_cop_summary_data_all["average_cop_rounded"]==cop)
        property_data_cfilt = pd.Series(property_data_all)[idx]
        # combine properties data for temp diff and cop:
        temp_data_all = np.array([])
        elec_data_all = np.array([])

        temp_data_all_filt = np.array([])
        elec_data_all_filt = np.array([])

        elec_band_data = pd.DataFrame()
        elec_band_data_filt = pd.DataFrame()
        for property_data in property_data_cfilt:
            # drop NaNs:
            property_data = property_data.dropna()

            # property_data_tfilt = property_data[property_data["daily_average_temperature_diff"] <= -5]
            # property_data_tfilt = property_data_tfilt[property_data_tfilt["daily_average_total_energy_consumed_H3"] >= 1]

            # temp_data_filt = np.array(property_data_tfilt["daily_average_temperature_diff"].values)
            # elec_data_filt = np.array(property_data_tfilt["daily_average_total_energy_consumed_H3"].values)

            temp_data = np.array(property_data["daily_average_temperature_diff"].values)
            elec_data = np.array(property_data["daily_average_total_energy_consumed_H3"].values)

            temp_data_all = np.concatenate((temp_data_all,temp_data))
            elec_data_all = np.concatenate((elec_data_all,elec_data))

            # temp_data_all_filt = np.concatenate((temp_data_all_filt,temp_data_filt))
            # elec_data_all_filt = np.concatenate((elec_data_all_filt,elec_data_filt))
        
        elec_band_data["temp_data_all"] = temp_data_all
        elec_band_data["elec_data_all"] = elec_data_all
        cop_band_data_all.append(elec_band_data)

        # elec_band_data_filt["temp_data_all"] = temp_data_all_filt
        # elec_band_data_filt["elec_data_all"] = elec_data_all_filt
        # cop_band_data_all_filt.append(elec_band_data_filt)

        (model_coeffs,fitted_model) = FitPolyModel(temp_data_all,elec_data_all,1)
        # np.savez(f"/ashp_model_cop_{cop}.npz",model_coeffs=model_coeffs)
        # fitted_model_filt = FitPolyModel(temp_data_all_filt,elec_data_all_filt,1)
        fitted_models.append(fitted_model)
        # fitted_models_filt.append(fitted_model_filt)

    poly_models["COP"] = cop_range
    poly_models["fitted_models"] = fitted_models
    # poly_models["fitted_models_filt"] = fitted_models_filt
    poly_models["cop_band_data_all"] = cop_band_data_all
    # poly_models["cop_band_data_all_filt"] = cop_band_data_all_filt
    return (poly_models)

(ashp_property_cop_summary_data_all) = ASHP_CopSummaryData(ashp_property_ids_accepted,ashp_property_data_all)
# (ashp_cop_vs_tempdiff_models) = ASHP_LinearRegressionCopVsTempDiff(ashp_property_ids_accepted,ashp_property_cop_summary_data_all,ashp_property_data_all)
(ashp_elec_vs_tempdiff_models) = ASHP_LinearRegressionEnergyVsTempDiff(ashp_property_ids_accepted,ashp_property_cop_summary_data_all,ashp_property_data_all)

##### Visualisation:

In [ ]:
def CreatePlots(property_ids,property_data_all):
    i=0
    # plot random selection of n properties to plot:
    n = 5
    indexes = list(range(len(property_ids)))
    idx = random.sample(indexes,n)
    property_ids_to_plot = [property_ids[i] for i in idx]
    property_ids_to_plot.append("EOH0018")
    for property_id in property_ids_to_plot:
        if(i==2):
            property_data = property_data_all[i]
            PlotDailyAveragesVsTime(property_id,property_data)
        i+=1
    PlotAllCopVsTempDiff(property_data_all)
    PlotCopVsTempModelsForCopBands(ashp_property_data_all,ashp_cop_vs_tempdiff_models)
    PlotElecVsTempModelsForCopBands(ashp_property_data_all,ashp_elec_vs_tempdiff_models)

def PlotPolyModelForCopBand(x_plot,x_var_name,y_var_name,xlabel,ylabel,cop,cop_data,cop_band_model):
    plt.scatter(cop_data[x_var_name],cop_data[y_var_name],label="Measured Data")
    plt.plot(x_plot, cop_band_model(x_plot),color="purple",label=f"Fitted Model")
    # plt.title(f"Polynomial Model for COP band {cop}")
    print(f"Polynomial Model for COP band {cop}")
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.ylim(top=100,bottom=0)
    plt.xlim(left=-25,right=5)
    # plt.axhline(y=3,color="r",linestyle="dashed",label="\"Standing\" Usage")
    plt.legend(loc=1)

def PlotLinearModelForCopBand(x_plot,x_var_name,y_var_name,xlabel,ylabel,cop,cop_data,cop_data_filt,cop_band_model):
    plt.scatter(cop_data[x_var_name],cop_data[y_var_name],color="orange",label="Data Rejected from Average COP Calculation")
    plt.scatter(cop_data_filt[x_var_name],cop_data_filt[y_var_name],label="Data Included in Average COP Calculation")
    # plt.plot(x_plot, cop_band_model(x_plot),color="purple")
    plt.axhline(y = cop+0.5, color = 'r', linestyle = 'dashed',label=f"COP Band Maximum Inclusion Limit") 
    plt.axhline(y = cop-0.5, color = 'r', linestyle = ':',label=f"COP Band Minimum Inclusion Limit") 
    # plt.title(f"Linear Model for COP band {cop}")
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.ylim(top=6,bottom=1)
    plt.xlim(left=-25,right=5)
    plt.legend(loc=1)

def PlotCopVsTempModelsForCopBands(ashp_property_data_all,cop_vs_tempdiff_models):
    # initialise plots:
    cop_bands_len = len(cop_vs_tempdiff_models["COP"])
    rows = 2*round(cop_bands_len/2)
    cols = 2
    # plt.subplots(nrows=rows, ncols=cols, figsize=(12,4*rows))
    plots_opacity = 0.7
    hpad = 2
    wpad = 0.5
    
    # data:
    cops = cop_vs_tempdiff_models["COP"]
    cop_data_all = cop_vs_tempdiff_models["cop_band_data_all"]
    cop_data_all_filt = cop_vs_tempdiff_models["cop_band_data_all_filt"]
    # cop_models_all = cop_vs_tempdiff_models["fitted_models"]
    cop_models_all_filt = cop_vs_tempdiff_models["fitted_models_filt"]
    
    # plot set 1 cop vs temp diff:
    for idx in cop_vs_tempdiff_models.index:
        # plot data:
        cop = cops[idx]
        cop_data = cop_data_all[idx]
        cop_data_filt = cop_data_all_filt[idx]
        # x_plot = np.linspace(min(cop_data["temp_data_all"]),max(cop_data["temp_data_all"]),500)
        x_plot = np.linspace(-25,5,500)
        cop_band_model = cop_models_all_filt[idx] 
        # cop_band_model_filt = cop_models_all_filt[idx] 

        # plt.subplot(rows,cols,2*idx+1)
        # PlotPolyModelForCopBand(x_plot,"temp_data_all","cop_data_all","Temperature (degrees C)","COP",cop,cop_data,cop_band_model)
        # plt.subplot(rows,cols,2*idx+2)
        plt.figure(figsize=(12,6))
        PlotLinearModelForCopBand(x_plot,"temp_data_all","cop_data_all","(External - Internal) Air Temperature Difference (K)","Daily Average COP",cop,cop_data,cop_data_filt,cop_band_model)

    # show plots 1:
    # plt.suptitle(f"Models Fitted Using Polynomial Regression for COP against Temperature Difference")
    plt.tight_layout(h_pad=hpad,w_pad=wpad,pad=hpad)
    plt.show()

def PlotElecVsTempModelsForCopBands(ashp_property_data_all,elec_vs_tempdiff_models):
    # initialise plots:
    cop_bands_len = len(elec_vs_tempdiff_models["COP"])
    rows = 2*round(cop_bands_len/2)
    cols = 2
    # plt.subplots(nrows=rows, ncols=cols, figsize=(12,4*rows))
    plots_opacity = 0.7
    hpad = 2
    wpad = 0.5
    
    # data:
    cops = elec_vs_tempdiff_models["COP"]
    cop_data_all = elec_vs_tempdiff_models["cop_band_data_all"]
    # cop_data_all_filt = elec_vs_tempdiff_models["cop_band_data_all_filt"]
    cop_models_all = elec_vs_tempdiff_models["fitted_models"]
    # cop_models_all_filt = elec_vs_tempdiff_models["fitted_models_filt"]
    
    # plot set 1 cop vs temp diff:
    for idx in elec_vs_tempdiff_models.index:
        # plot data:
        cop = cops[idx]
        cop_data = cop_data_all[idx]
        # cop_data_filt = cop_data_all_filt[idx]
        x_plot = np.linspace(-25,5,500)
        # x_plot = np.linspace(min(cop_data["temp_data_all"]),max(cop_data["temp_data_all"]),500)
        cop_band_model = cop_models_all[idx] 
        # cop_band_model_filt = cop_models_all_filt[idx] 

        # plt.subplot(rows,cols,2*idx+1)
        # plt.ylim(top=100,bottom=None)
        # PlotPolyModelForCopBand(x_plot,"temp_data_all","elec_data_all","Temperature Difference (degrees C)","Consumption (kWh)",cop,cop_data,cop_band_model)
        # plt.subplot(rows,cols,2*idx+2)
        plt.figure(figsize=(12,6))
        plt.ylim(top=100,bottom=None)
        # PlotLinearModelForCopBand(x_plot,"temp_data_all","elec_data_all","Temperature Difference (degrees C)","Consumption (kWh)",cop,cop_data,cop_data_filt,cop_band_model_filt)
        PlotPolyModelForCopBand(x_plot,"temp_data_all","elec_data_all","(External - Internal) Air Temperature Difference (K)","Daily Average Electricity Consumption (kWh)",cop,cop_data,cop_band_model)

    # show plots 1:
    # plt.suptitle(f"Models Fitted Using Polynomial Regression for Energy Consumed (kWh) against Temperature Difference")
    plt.tight_layout(h_pad=hpad,w_pad=wpad,pad=hpad)
    plt.show()    

def PlotDailyAveragesVsTime(property_id,property_data):
    # initialise plots:
    rows = 4
    cols = 2
    # plt.subplots(nrows=rows, ncols=cols, figsize=(18,6*rows))
    # plt.suptitle(f"Property Data Overview and Metadata Initial Analysis")
    plots_opacity = 1
    hpad = 2
    wpad = 0.5

    a = 12
    b = 6
    size=20
    # plot temperatures:
    # plt.subplot(rows,cols,1)
    plt.figure(figsize=(a,b))
    plt.scatter(property_data["start_of_day"],property_data["daily_average_external_temperature"],
                label="Daily Average External Air Temperature",alpha=plots_opacity,s=15)
    plt.scatter(property_data["start_of_day"],property_data["daily_average_internal_temperature"],
                label="Daily Average Internal Air Temperature",alpha=plots_opacity,s=15)
    # plt.title("Daily Average Temperatures - May 2021 to September 2022")
    plt.xlabel("Date yyyy-mm")
    plt.ylabel('Temperature (degrees C)')
    plt.legend(loc=1)
    plt.ylim(top=30,bottom=-5)
    plt.xticks(rotation=90)

    # plt.subplot(rows,cols,2)
    # plt.scatter(property_data["start_of_day"],property_data["daily_average_total_energy_consumed_H3"],
    #             label="daily_average_total_energy_consumed_H3",alpha=plots_opacity)
    # plt.scatter(property_data["start_of_day"],property_data["daily_average_total_energy_consumed_H4"],
    #             label="daily_average_total_energy_consumed_H4",alpha=plots_opacity)
    # plt.title("Daily Average System Energy Consumed H3 vs H4 Over 1 Year")
    # plt.xlabel("Date yyyy-mm")
    # plt.ylabel('Energy (kWh)')
    # plt.legend(loc=1)
    # plt.xticks(rotation=90)

    # plot energies for system boundary H3:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,3)
    plt.scatter(property_data["start_of_day"],property_data["daily_average_total_energy_consumed_H3"],
                label="Daily Average Total Electrical Energy Consumed",alpha=plots_opacity,s=size)
    plt.scatter(property_data["start_of_day"],property_data["daily_average_heating_energy_output"],
                label="Daily Average Total Heat Energy Output",alpha=plots_opacity,s=size)
    # plt.title("Daily Average Values for Heat Energy Output and Electrical Energy Consumed")
    plt.xlabel("Date yy-mm")
    plt.ylabel('Average Total Energy (kWh)')
    plt.ylim(top=120,bottom=0)
    plt.legend(loc=1)
    plt.xticks(rotation=90)

    # plot energies for system boundary H3:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,3)
    plt.scatter(property_data["daily_average_total_energy_consumed_H3"],property_data["daily_average_heating_energy_output"],alpha=plots_opacity,s=size)
    # plt.title("Daily Average Values for Heat Energy Output and Electrical Energy Consumed")
    plt.ylabel('Daily Average Total Heat Energy Output (kWh)')
    plt.xlabel('Daily Average Total Electrical Energy Consumed (kWh)')
    plt.ylim(top=120,bottom=0)
    plt.xlim(left=0,right=50)

    # plot energies for system boundary H3:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,3)
    plt.scatter(property_data["daily_average_total_energy_consumed_H3"],property_data["daily_average_heating_energy_output"],alpha=plots_opacity,s=size)
    # plt.title("Daily Average Values for Heat Energy Output and Electrical Energy Consumed")
    plt.ylabel('Daily Average Total Heat Energy Output (kWh)')
    plt.xlabel('Daily Average Total Electrical Energy Consumed (kWh)')
    plt.ylim(top=120,bottom=0)
    plt.xlim(left=0,right=50)
    xdata = np.linspace(0,50,10)
    property_data = property_data.dropna()
    # property_data = property_data[property_data["daily_average_heating_energy_output"]>20]
    x = property_data["daily_average_total_energy_consumed_H3"]
    y = property_data["daily_average_heating_energy_output"]
    coeffs = np.polyfit(x,y, deg=1)
    poly_model = np.poly1d(coeffs)
    plt.plot(xdata,poly_model(xdata),'--',color="purple",label=f"Linear Model Fitted: y = {round(coeffs[0],2)}x + {round(coeffs[1],2)}")
    # plt.text(x=35,y=5,s=f"Linear Model Fitted: y = {round(coeffs[0],2)}x + {round(coeffs[1],2)}")
    # plt.axhline(y=20, color = 'r', linestyle = ':',label="")
    plt.legend(loc=1)

    # plot COP for system boundary H3:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,4)
    plt.scatter(property_data["start_of_day"],property_data["daily_cop_H3"],
                label="daily_cop_H3",alpha=plots_opacity,s=size)
    # plt.title("Daily Average COP Values")
    plt.xlabel("Date yyyy-mm")
    plt.ylabel('COP')
    plt.ylim(top=4,bottom=0)
    # plt.legend(loc=1)
    plt.xticks(rotation=90)

    # plot COP vs external temp for system boundary H4:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,5)
    plt.scatter(property_data["daily_average_external_temperature"],property_data["daily_cop_H3"],alpha=plots_opacity,s=size)
    # plt.scatter(property_data["daily_average_external_temperature"],property_data["daily_cop_H4"],
    #             label="daily_cop_H4",alpha=plots_opacity)
    # plt.title("Daily Average COP with Varying External Temperature")
    plt.xlabel('External Air Temperature (degrees C)')
    plt.ylabel('Daily Average COP')
    # plt.legend(loc=1)
    plt.ylim(top=4,bottom=0)

    # plot COP vs (internal - external temp) for system boundary H4:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,6)
    plt.scatter(property_data["daily_average_temperature_diff"],property_data["daily_cop_H3"],
                label="daily_cop_H3",alpha=plots_opacity,s=size)
    # plt.scatter(property_data["daily_average_temperature_diff"],property_data["daily_cop_H4"],
    #             label="daily_cop_H4",alpha=plots_opacity)
    # plt.title("Daily Average COP with Varying (External - Internal) Air Temperature Difference")
    plt.xlabel('(External - Internal) Air Temperature Difference (K)')
    plt.ylabel('Daily Average COP')
    plt.ylim(top=4,bottom=0)
    # plt.legend(loc=1)

    # # plot elec vs external temp for system boundary H4:
    # plt.subplot(rows,cols,7)
    # plt.scatter(property_data["daily_average_external_temperature"],property_data["daily_average_total_energy_consumed_H3"],
    #             label="daily_average_consumtion_H3",alpha=plots_opacity)
    # plt.scatter(property_data["daily_average_external_temperature"],property_data["daily_average_total_energy_consumed_H4"],
    #             label="daily_average_consumtion_H4",alpha=plots_opacity)
    # plt.title("Daily Average Energy Consumed vs External Temperature for System")
    # plt.xlabel('Temperature (degrees C)')
    # plt.ylabel('Energy Consumed (kWh)')
    # plt.legend()

    # plot elec vs (internal - external temp) for system boundary H4:
    plt.figure(figsize=(a,b))
    # plt.subplot(rows,cols,8)
    plt.scatter(property_data["daily_average_temperature_diff"],property_data["daily_average_total_energy_consumed_H3"],
                label="daily_average_consumtion_H3",alpha=plots_opacity,s=25)
    # plt.scatter(property_data["daily_average_temperature_diff"],property_data["daily_average_total_energy_consumed_H4"],
    #             label="daily_average_consumtion_H4",alpha=plots_opacity)
    # plt.title("Daily Average Energy Consumed for Varying Air Temperature Difference")
    plt.xlabel('(External - Internal) Air Temperature Difference (K)')
    plt.ylabel('Daily Average Total Electrical Energy Consumed (kWh)')
    plt.ylim(top=50,bottom=0)
    # plt.legend()

    # show plots:
    plt.tight_layout(h_pad=hpad,w_pad=wpad)
    plt.show()

def PlotAllCopVsTempDiff(property_data_all):
    plots_opacity = 0.7
    plt.plot(figsize=(12,4))
    for property_data in property_data_all:
        plt.scatter(property_data["daily_average_temperature_diff"],property_data["daily_cop_H3"],
                label="daily_cop_H3",alpha=plots_opacity)
        plt.title("Daily COP vs Temperature Difference for System")
        plt.xlabel('Temperature Difference (degrees C)')
        plt.ylabel('COP')
        plt.xticks(rotation=90)

CreatePlots(ashp_property_ids_accepted,ashp_property_data_all)


### Appliance Load Data Analysis Code
#### Note: data to run this code is NOT provided. Published data can be downloaded from IDEAL: https://datashare.ed.ac.uk/handle/10283/3647

##### Code used to load data and filter for necessary files.

In [ ]:
# #loadin2
# def DL_GetPropertyData(property_id,file_properties_base,filepath):
#     # electricity mains:
#     folder = "".join([file_properties_base,"home",str(property_id)])
#     file_property = glob.glob(filepath + '*/*_electric-mains_electric-combined.csv', recursive=True)[0]
#     # try:
#     # read in time intervals of "interval" minutes:
#     interval = 5
#     # n = number of seconds in "interval" minutes:
#     n = (interval*60)
#     # column header names to use:
#     headers = ["timestamp", "electricity_reading_watts"]
#     print(filepath)
#     return pd.read_csv(file_property, skiprows = lambda row: row % n, names = headers)
#     # except:
#         # print(f"Failed to open file: {property_id}, at location: {filepath}.")
#         # return None
    
# def DL_LoadPropertyMetadataFromCsv(folder_location, filename):
#     file_property_ids = folder_location+filename
#     try:
#         file_pointer = open(file_property_ids, "r")
#     except:
#         print(f"Failed to open file: {file_property_ids}")
#     else:
#         # try: 
#             i = 0
#             property_ids = []
#             property_build_eras = []
#             property_occupants = []
#             for file_line in csv.reader(file_pointer):
#                 property_id = file_line[0]
#                 property_sensors_setup = file_line[1]
#                 # skip header:
#                 if not property_id.isnumeric():
#                     continue
#                 # choose to include / exclude "enhanced" / "standard" home sensor setup:
#                 # if str(property_sensors_setup) == "standard":
#                 #     continue
#                 # update valid property count:
#                 i+=1
#                 # # select every 3rd property to analyse: (IMPROVE THIS)
#                 # if i % 3 != 0:
#                 #     continue
#                 property_occupant = file_line[3]
#                 property_build_era = file_line[15]
#                 property_ids.append(property_id)
#                 property_build_eras.append(property_build_era)
#                 property_occupants.append(property_occupant)

#             property_metadata = pd.DataFrame()
#             property_metadata["property_ids"] = property_ids
#             property_metadata["property_build_eras"] = property_build_eras
#             property_metadata["property_occupants"] = property_occupants
#             property_metadata.drop(property_metadata.tail(2).index,inplace = True)
#             return property_metadata
#         # except: 
#         #     print(f"Failed to open file: {property_id}.")
    
# def DL_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data):
#     property_metadata = DL_LoadPropertyMetadataFromCsv(folder_location,append_location_property_ids)
#     if(property_metadata["property_ids"] is None):
#         print(f"Failed to load property ids from location {folder_location+append_location_property_ids}")
#     property_ids = property_metadata["property_ids"]
#     property_data_unprocessed_all = []
#     for filepath in os.scandir(folder_location_property_data):
#         property_id = (filepath.name).split("_")[0]
#         path = filepath.path
#         property_data_unprocessed = DL_GetPropertyData(property_id,folder_location_property_data,path)
#         property_data_unprocessed.to_pickle(folder_location_property_data+f"/property_data_{property_id}.pkl")
#         # property_data_unprocessed_all.append(property_data_unprocessed)
#     return (property_ids,property_metadata,property_data_unprocessed_all)
        

# folder_location = ""
# append_location_property_ids = 'DS_10283_3647/metadata_and_surveys/metadata/home.csv'
# # folder_location_property_data = ''
# folder_location_property_data = ''
# (dl_property_ids,dl_property_metadata,dl_property_data_unprocessed_all) = DL_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data)

##### Load Data from Files:

In [ ]:
def DL_GetPropertyData(property_id,folder_location_property_data):
    filepath = folder_location_property_data+f"property_data_home{property_id}.pkl"
    try:
        return pd.read_pickle(filepath)
    except:
        print(f"Failed to open file: {filepath}.")
        return None
    
def DL_LoadPropertyMetadataFromCsv(folder_location, filename):
    file_property_ids = folder_location+filename
    try:
        file_pointer = open(file_property_ids, "r")
    except:
        print(f"Failed to open file: {file_property_ids}")
    else:
        # try: 
            property_ids = []
            property_build_eras = []
            property_occupants = []
            for file_line in csv.reader(file_pointer):
                property_id = file_line[0]
                property_sensors_setup = file_line[1]
                # skip header:
                if not property_id.isnumeric():
                    continue
                # choose to include / exclude "enhanced" / "standard" home sensor setup:
                # if str(property_sensors_setup) == "standard":
                #     continue
                property_occupant = file_line[3]
                property_build_era = file_line[15]
                property_ids.append(property_id)
                property_build_eras.append(property_build_era)
                property_occupants.append(property_occupant)

            property_metadata = pd.DataFrame()
            property_metadata["property_ids"] = property_ids
            property_metadata["property_build_eras"] = property_build_eras
            property_metadata["property_occupants"] = property_occupants
            property_metadata.drop(property_metadata.tail(2).index,inplace = True)
            return property_metadata
        # except: 
        #     print(f"Failed to open file: {property_id}.")
    
def DL_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data):
    property_metadata = DL_LoadPropertyMetadataFromCsv(folder_location,append_location_property_ids)
    if(property_metadata["property_ids"] is None):
        print(f"Failed to load property ids from location {folder_location+append_location_property_ids}")
    property_ids = property_metadata["property_ids"]
    property_data_unprocessed_all = []
    for property_id in property_ids:
        property_data_unprocessed = DL_GetPropertyData(property_id,folder_location_property_data)
        property_data_unprocessed_all.append(property_data_unprocessed)
    print(len(property_data_unprocessed_all))
    return (property_ids,property_metadata,property_data_unprocessed_all)
        

folder_location = ""
append_location_property_ids = 'DS_10283_3647/metadata_and_surveys/metadata/home.csv'
folder_location_property_data = ''
(dl_property_ids,dl_property_metadata,dl_property_data_unprocessed_all) = DL_LoadPropertiesCsvData(folder_location,append_location_property_ids,folder_location_property_data)

##### Filter and Sort Data for Analysis

In [ ]:
def DL_RejectData(data,acceptable_percentage):
    # drop property where missing data >= 50%
    missing_points = np.count_nonzero(np.isnan(data))
    missing_percentage = 100*missing_points/len(data)
    return missing_percentage >= acceptable_percentage
    
def DL_ProcessData(property_data_unprocessed):
    if(property_data_unprocessed is None):
        return None
    # split data by day to find daily averages:
    # try:
    # data is recorded in intervals of:
    # convert timestamps to datetime format:
    time_format = "%Y-%m-%d %H:%M:%S"
    property_data_unprocessed["timestamp"] = pd.to_datetime(property_data_unprocessed["timestamp"],format=time_format)
    # drop property where missing data >= 50%
    if(DL_RejectData(property_data_unprocessed["electricity_reading_watts"],50)):
        print("missing data 1 reject")
        return None
    # calculate time difference between entries:
    property_data_unprocessed["timedeltas"] = property_data_unprocessed["timestamp"].diff()
    # calculate time differences in hours:
    property_data_unprocessed["timedeltas_hours"] = property_data_unprocessed["timedeltas"].dt.total_seconds()/(60*60)
    if(np.any(property_data_unprocessed["timedeltas_hours"] > 100*24)):
        print("timedelta reject")
        return None
    # drop days after missing data gaps where timedeltas_hours > 1.5(24) hours:
    max_hrs = 24*1.2
    property_data_unprocessed.drop(property_data_unprocessed[property_data_unprocessed["timedeltas_hours"] > max_hrs].index, inplace=True)
    # convert usage to kWh:
    property_data_unprocessed["electricity_reading_kwh"] = np.multiply((property_data_unprocessed["electricity_reading_watts"]/1000),property_data_unprocessed["timedeltas_hours"])
    # make timestamp the dataframe index:
    property_data_unprocessed.set_index("timestamp")
    # resample data by day:
    daily_cumulative_electricity_reading_kwh = property_data_unprocessed.resample('D',on="timestamp").sum()
    # drop incomplete days where time < 18 hours:
    daily_cumulative_electricity_reading_kwh.drop(daily_cumulative_electricity_reading_kwh[daily_cumulative_electricity_reading_kwh["timedeltas_hours"] < 18].index, inplace=True)
    # calculate months usage (kWh) - resample by month:
    monthly_cumulative_electricity_reading_kwh = property_data_unprocessed.resample('M',on="timestamp").sum()

    # add data to new dataframe:
    property_data = pd.DataFrame(daily_cumulative_electricity_reading_kwh["electricity_reading_kwh"])
    property_data["monthly_c_electricity_reading_kwh"] = monthly_cumulative_electricity_reading_kwh["electricity_reading_kwh"]
    # print(property_data)
    # calculate flat average for consumption (kWh):
    average_usage_flat_kwh = np.mean(property_data["electricity_reading_kwh"])

    # calculate flat average for montly consumption (kWh):
    montly_usages = pd.DataFrame()
    montly_usages["monthly_c_electricity_reading_kwh"] = property_data["monthly_c_electricity_reading_kwh"].dropna()
    time_data = pd.Series(montly_usages.index)
    time_data_months = time_data.apply(lambda date: pd.Timestamp(date).month)
    montly_usages["months"] = time_data_months.values
    montly_usages = montly_usages.set_index("months")
    # print(montly_usages)
    average_monthly_usage_kwh = pd.DataFrame()
    usages = []
    months = []
    for m in range(1,13):
        months_all = montly_usages[montly_usages.index==m]
        av = np.mean(months_all)
        months.append(m)
        usages.append(av)
        # print(av)
    average_monthly_usage_kwh["month"] = months
    average_monthly_usage_kwh["usage_kwh"] = usages

    # calculate flat average for daily consumption (kWh):
    daily_usages = pd.DataFrame()
    daily_reading_dropna = property_data["electricity_reading_kwh"].dropna()
    daily_reading_drop0 = daily_reading_dropna[daily_reading_dropna > 0]
    daily_usages["daily_c_electricity_reading_kwh"] = daily_reading_drop0

    time_data = pd.Series(daily_usages.index)
    time_data_days = time_data.apply(lambda date: pd.Timestamp(date).day_of_year)
    daily_usages["day"] = time_data_days.values
    daily_usages = daily_usages.set_index("day")
    # print(daily_usages)
    average_daily_usage_kwh = pd.DataFrame()
    usages = []
    days = []
    for d in range(1,366):
        days_all = daily_usages[daily_usages.index==d]
        av = np.mean(days_all)
        days.append(d)
        usages.append(av)
        # print(av)
    average_daily_usage_kwh["day"] = days
    average_daily_usage_kwh["usage_kwh"] = usages
    average_daily_usage_kwh["occ"] = 999

    # print(average_monthly_usage_kwh)
    # print(average_monthly_usage_kwh)
    # drop property where missing data >= 50%
    if(DL_RejectData(property_data["electricity_reading_kwh"],50)):
        print("missing data 2 reject")
        return None
    return (property_data,average_usage_flat_kwh,average_monthly_usage_kwh,average_daily_usage_kwh)
    # except:
        # print("exception")
        # return None

def DL_ProcessPropertiesData(property_ids,property_metadata,property_data_unprocessed_all):
    property_ids_accepted = []
    property_data_all = []
    average_usages_flat_kwh = []
    average_monthly_usages_kwh = pd.DataFrame()
    average_daily_usages_kwh = pd.DataFrame()
    i=0
    for property_id in property_ids:
        property_data_unprocessed = property_data_unprocessed_all[i]
        property_occupants = property_metadata["property_occupants"][i]
        i+=1
        process_result = DL_ProcessData(property_data_unprocessed)
        if(process_result is None):
            print(f"Property {property_id} rejected from analysis")
            property_ids_accepted.append(property_id)
            property_data_all.append(None)
            average_usages_flat_kwh.append(None)
            average_monthly_usages_kwh = pd.concat([average_monthly_usages_kwh,None])
            average_daily_usages_kwh = pd.concat([average_daily_usages_kwh,None])
        else:
            (property_data,average_usage_flat_kwh,average_monthly_usage_kwh,average_daily_usage_kwh) = process_result
            property_ids_accepted.append(property_id)
            property_data_all.append(property_data)
            average_usages_flat_kwh.append(average_usage_flat_kwh)
            average_monthly_usages_kwh = pd.concat([average_monthly_usages_kwh,average_monthly_usage_kwh])
            average_daily_usage_kwh["occ"] = property_occupants
            average_daily_usages_kwh = pd.concat([average_daily_usages_kwh,average_daily_usage_kwh])

    property_metadata["property_occupants"] = [int(i) for i in property_metadata["property_occupants"]]
    property_metadata["average_usage_flat_kwh"] = average_usages_flat_kwh

    average_daily_usages_kwh_combined = pd.DataFrame()
    occ_range = average_daily_usages_kwh["occ"].unique()

    for occ in occ_range:
        average_daily_usage_kwh_combined = pd.DataFrame()
        # ids = property_cop_summary_data_all[property_cop_summary_data_all["average_cop_rounded"]==cop]["property_id"]
        # idx = np.array(dl_property_metadata["property_occupants"]==occ)
        dl_average_daily_usages_kwh_grouped = average_daily_usages_kwh.groupby(average_daily_usages_kwh.occ)
        dl_average_daily_usages_kwh_filt = dl_average_daily_usages_kwh_grouped.get_group(occ)
        average_daily_usages_kwh_idx = dl_average_daily_usages_kwh_filt.set_index("day")
        usages = []
        days = []
        for d in range(1,366):
            days_all = average_daily_usages_kwh_idx[average_daily_usages_kwh_idx.index==d]
            av = np.mean(days_all)
            days.append(d)
            usages.append(av)
        average_daily_usage_kwh_combined["day"] = days
        average_daily_usage_kwh_combined["usage_kwh"] = usages
        average_daily_usage_kwh_combined["occ"] = occ
        average_daily_usages_kwh_combined = pd.concat([average_daily_usages_kwh_combined,average_daily_usage_kwh_combined])

    # property_metadata["average_monthly_usage_kwh"] = average_monthly_usage_kwh["usage_kwh"]
    # print(property_metadata["average_monthly_usage_kwh"][0])
    return (property_ids_accepted, property_metadata, property_data_all, average_monthly_usages_kwh,average_daily_usages_kwh,average_daily_usages_kwh_combined)

(dl_property_ids_accepted,dl_property_metadata,dl_property_data_all,dl_monthly_averages_kwh,dl_average_daily_usages_kwh,dl_average_daily_usages_kwh_combined) = DL_ProcessPropertiesData(dl_property_ids,dl_property_metadata,dl_property_data_unprocessed_all)
print(dl_average_daily_usages_kwh_combined[1:370])


##### Data Analysis and Calculation of Metrics

In [ ]:
def FitPolyModel(time_data_all,elec_data_all,degree):
    # remove any rows with NaN values:
    model_coeffs = np.polyfit(time_data_all, elec_data_all, deg=degree)
    poly_model = np.poly1d(model_coeffs)
    return (model_coeffs, poly_model)

def DL_LinearRegressionEnergyVsDate2(dl_property_ids,dl_property_metadata,dl_property_data_all,dl_average_daily_usages_kwh,dl_average_daily_usages_kwh_combined):
    # distinct occupants:
    occ_range = dl_average_daily_usages_kwh_combined["occ"].unique()
    occ_range.sort()
    occ_range = occ_range[occ_range >= 1]

    # polynomial regression model for each occ:
    models_data = pd.DataFrame()
    data_all = []
    fitted_models = []
    for occ in occ_range:
        dl_average_daily_usages_kwh_grouped = dl_average_daily_usages_kwh_combined.groupby(dl_average_daily_usages_kwh_combined.occ)
        dl_average_daily_usages_kwh_filt = dl_average_daily_usages_kwh_grouped.get_group(occ)

        print(dl_average_daily_usages_kwh_filt)
        
        day_of_year = dl_average_daily_usages_kwh_filt["day"]
        usage_kwh = dl_average_daily_usages_kwh_filt["usage_kwh"]
        (model_coeffs, fitted_model) = FitPolyModel(day_of_year,usage_kwh,2)
        # np.savez(f"/dl_model_occupant_{occ}.npz",model_coeffs=model_coeffs)
        fitted_models.append(fitted_model)
        

        data_all.append(dl_average_daily_usages_kwh_filt)

    models_data["property_occupants"] = occ_range
    models_data["fitted_models"] = fitted_models
    models_data["occ_data_all"] = data_all
    return (models_data)

def DL_LinearRegressionEnergyVsDate(dl_property_ids,dl_property_metadata,dl_property_data_all,dl_average_daily_usages_kwh,dl_average_daily_usages_kwh_combined):
    # distinct occupants:
    # occ_range = dl_property_metadata["property_occupants"].unique()
    occ_range = dl_average_daily_usages_kwh["occ"].unique()
    occ_range.sort()
    occ_range = occ_range[occ_range >= 1]

    # polynomial regression model for each occ:
    models_data = pd.DataFrame()
    data_all = []
    fitted_models = []
    j = 0
    for occ in occ_range:
        dl_average_daily_usages_kwh_grouped = dl_average_daily_usages_kwh.groupby(dl_average_daily_usages_kwh.occ)
        dl_average_daily_usages_kwh_filt = dl_average_daily_usages_kwh_grouped.get_group(occ)
        print(dl_average_daily_usages_kwh_filt)

        data_all.append(dl_average_daily_usages_kwh_filt)

        day_of_year = dl_average_daily_usages_kwh_filt["day"]
        usage_kwh = dl_average_daily_usages_kwh_filt["usage_kwh"]
        fitted_model = FitPolyModel(day_of_year,usage_kwh,2)
        fitted_models.append(fitted_model)

    models_data["property_occupants"] = occ_range
    models_data["fitted_models"] = fitted_models
    models_data["occ_data_all"] = data_all
    return (models_data)

(dl_elec_vs_date_models) = DL_LinearRegressionEnergyVsDate2(dl_property_ids_accepted,dl_property_metadata,dl_property_data_all,dl_average_daily_usages_kwh,dl_average_daily_usages_kwh_combined)
print(dl_elec_vs_date_models)

##### Visualisation

In [ ]:
def CreatePlots(property_ids_accepted, property_metadata, dl_elec_vs_date_models, property_data_all,dl_monthly_averages_kwh,dl_average_daily_usages_kwh_combined):
    PlotDailyAveragesVsTime(property_ids_accepted,property_data_all)
    PlotMetaData2(property_metadata,dl_monthly_averages_kwh)
    # PlotModel(dl_models_data, dl_fitted_model)
    PlotElecVsTempModelsForOccBands(dl_elec_vs_date_models,dl_average_daily_usages_kwh_combined)

def PlotMetaData2(property_metadata,dl_monthly_averages_kwh):
    plots_opacity = 0.7
    hpad = 2
    wpad = 1
    # plot daily average electricity usage (kWh) vs building era:
    # re order x axis:
    property_metadata_by_era = pd.DataFrame()
    property_metadata_by_era["property_build_eras"] = property_metadata["property_build_eras"]
    property_metadata_by_era["average_usage_flat_kwh"] = property_metadata["average_usage_flat_kwh"]
    dates_all = []
    for dates in property_metadata["property_build_eras"]:
        if(dates[0] == "B"):
            date = pd.to_datetime("1849",format="%Y")
        elif(dates[-1] == "r"):
            date = pd.to_datetime(str(dates).split(" ")[0],format="%Y")
        else:
            date = pd.to_datetime(str(dates).split("-")[0],format="%Y")
        dates_all.append(date)
    property_metadata_by_era["property_build_eras_start_date"] = dates_all

    property_metadata_by_era.sort_values(by=['property_build_eras_start_date'], inplace=True)

    # plot daily average electricity usage (kWh) vs build era:
    plt.figure(figsize=(12,6))
    # plt.subplot(rows,cols,1)
    plt.scatter(property_metadata_by_era["property_build_eras"],property_metadata_by_era["average_usage_flat_kwh"],
                alpha=plots_opacity)
    # plt.set_title("Building Era vs Average Daily Electricity Consumption (kWh)")
    plt.ylabel('Average Daily Total Electricity Consumption (kWh)')
    plt.xlabel("Build Era of Property")
    plt.xticks(rotation=90)

     
    # plot daily average electricity usage (kWh) vs occupants:
    plt.figure(figsize=(12,6))
    # plt.subplot(rows,cols,2)
    plt.scatter(property_metadata["property_occupants"],property_metadata["average_usage_flat_kwh"],
                alpha=plots_opacity)
    # plt.title("Number of Occupants vs Average Daily Electricity Consumption (kWh)")
    plt.ylabel('Average Daily Total Electricity Consumption (kWh)')
    plt.xlabel("Number of Occupants")

    # plot average daily average electricity usage (kWh):
    # data in format for box plot:
    box_data = []
    box_data_labels = []
    box_data_labels_ticks = []
    property_metadata_by_era_split = property_metadata_by_era.groupby(property_metadata_by_era.property_build_eras)
    i = 1
    for era in property_metadata_by_era["property_build_eras"].unique():
        era_data = property_metadata_by_era_split.get_group(str(era))["average_usage_flat_kwh"]
        era_data = era_data.dropna()
        box_data.append(era_data)
        box_data_labels.append(str(era))
        box_data_labels_ticks.append(i)
        i+=1
        
    # plt.subplot(rows,cols,3)
    plt.figure(figsize=(12,6))
    plt.boxplot(box_data)
    # plt.title("Mean and Spread for Build Era vs Consumption (kWh)")
    plt.ylabel('Average Daily Total Electricity Consumption (kWh)')
    plt.xlabel("Build Era of Property")
    plt.xticks(ticks=box_data_labels_ticks,labels=box_data_labels)
    plt.xticks(rotation=90)

    # plot average daily average electricity usage (kWh):
    # data in format for box plot:
    box_data = []
    box_data_labels = []
    box_data_labels_ticks = []
    property_metadata.sort_values(by=['property_occupants'], inplace=True)
    property_metadata_by_occ_split = property_metadata.groupby(property_metadata.property_occupants)
    i = 1
    for occ in property_metadata["property_occupants"].unique():
        occ_data = property_metadata_by_occ_split.get_group(occ)["average_usage_flat_kwh"]
        occ_data = occ_data.dropna()
        box_data.append(occ_data)
        box_data_labels.append(str(occ))
        box_data_labels_ticks.append(i)
        i+=1

    # plot average daily average electricity usage (kWh) vs occupants:
    # plt.subplot(rows,cols,4)
    plt.figure(figsize=(12,6))
    plt.boxplot(box_data)
    # plt.title("Mean and Spread of  Number of Average Daily Electricity Consumption for Number of Occupants")
    plt.ylabel('Average Daily Total Electricity Consumption (kWh)')
    plt.xlabel("Number of Occupants")
    plt.xticks(ticks=box_data_labels_ticks,labels=box_data_labels)
    

    # plot average monthly usages for occupants as bars:
    plt.figure(figsize=(12,5))
    # plt.subplot(3,1,3)

    # montly averages for occupants:
    month_avs = []
    dl_monthly_averages_kwh.sort_values(by=['month'], inplace=True)
    monthly_averages_kwh = dl_monthly_averages_kwh.groupby(dl_monthly_averages_kwh.month)
    for m in dl_monthly_averages_kwh["month"].unique():
        month_data = monthly_averages_kwh.get_group(m)["usage_kwh"]
        month_data = month_data.dropna()
        av = np.mean(month_data)
        month_avs.append(av)

    xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    bars = plt.bar(xlabels, month_avs, label ='average monthly usage (kWh)')
    plt.bar_label(bars, fmt='{:,.0f}')
    plt.xlabel('Month of Year') 
    plt.ylabel('Average Monthly Total Electricity Consumption (kWh)') 
    # plt.title('Total Monthly Electricity Consumption for Each Month (kWh)') 


    # plot average daily usages for months as bars:
    plt.figure(figsize=(12,5))
    # plt.subplot(3,1,3)

    # daily averages for occupants:
    month_avs = []
    dl_monthly_averages_kwh.sort_values(by=['month'], inplace=True)
    monthly_averages_kwh = dl_monthly_averages_kwh.groupby(dl_monthly_averages_kwh.month)
    for m in dl_monthly_averages_kwh["month"].unique():
        month_data = monthly_averages_kwh.get_group(m)["usage_kwh"]
        month_data = month_data.dropna()
        days = [0,31,28,31,30,31,30,31,31,30,31,30,31]
        av = np.mean(month_data)/days[m]
        month_avs.append(av)

    xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    bars = plt.bar(xlabels, month_avs, label ='average monthly usage (kWh)')
    plt.xlabel('Month of Year') 
    plt.ylabel('Average Daily Total Electricity Consumption (kWh)') 
    # plt.title('Average Daily Total Electricity Consumption for Each Month') 
    plt.bar_label(bars, fmt='{:,.2f}')


    # show plots:
    plt.tight_layout(h_pad=hpad,w_pad=wpad)
    plt.show()

def PlotMetaData(property_metadata,dl_monthly_averages_kwh):
    # initialise plots:

    rows = 4
    cols = 2
    # fig = plt.subplots(figsize=(12,4*rows))
    fig = plt.figure(figsize=(12,4*rows),constrained_layout=True)
    gs = GridSpec(rows,cols,figure=fig)
    plt.suptitle(f"Analysis of Metadata and Factors Affecting Energy Consumption")
    plots_opacity = 0.7
    hpad = 2
    wpad = 1

    # plot daily average electricity usage (kWh) vs building era:
    # re order x axis:
    property_metadata_by_era = pd.DataFrame()
    property_metadata_by_era["property_build_eras"] = property_metadata["property_build_eras"]
    property_metadata_by_era["average_usage_flat_kwh"] = property_metadata["average_usage_flat_kwh"]
    dates_all = []
    for dates in property_metadata["property_build_eras"]:
        if(dates[0] == "B"):
            date = pd.to_datetime("1849",format="%Y")
        elif(dates[-1] == "r"):
            date = pd.to_datetime(str(dates).split(" ")[0],format="%Y")
        else:
            date = pd.to_datetime(str(dates).split("-")[0],format="%Y")
        dates_all.append(date)
    property_metadata_by_era["property_build_eras_start_date"] = dates_all

    property_metadata_by_era.sort_values(by=['property_build_eras_start_date'], inplace=True)

    # plot daily average electricity usage (kWh) vs build era:
    ax1 = fig.add_subplot(gs[0,0])
    # plt.subplot(rows,cols,1)
    ax1.scatter(property_metadata_by_era["property_build_eras"],property_metadata_by_era["average_usage_flat_kwh"],
                alpha=plots_opacity)
    ax1.set_title("Building Era vs Average Daily Electricity Consumption (kWh)")
    ax1.set_ylabel('Daily Electricity Consumption (kWh)')
    ax1.set_xlabel("Build Era of Property")
    plt.xticks(rotation=90)

     
    # plot daily average electricity usage (kWh) vs occupants:
    ax2 = fig.add_subplot(gs[0,1])
    # plt.subplot(rows,cols,2)
    ax2.scatter(property_metadata["property_occupants"],property_metadata["average_usage_flat_kwh"],
                alpha=plots_opacity)
    ax2.set_title("Number of Occupants vs Average Daily Electricity Consumption (kWh)")
    ax2.set_ylabel('Daily Electricity Consumption (kWh)')
    ax2.set_xlabel("Number of Occupants")

    # plot average daily average electricity usage (kWh):
    # data in format for box plot:
    box_data = []
    box_data_labels = []
    box_data_labels_ticks = []
    property_metadata_by_era_split = property_metadata_by_era.groupby(property_metadata_by_era.property_build_eras)
    i = 1
    for era in property_metadata_by_era["property_build_eras"].unique():
        era_data = property_metadata_by_era_split.get_group(str(era))["average_usage_flat_kwh"]
        era_data = era_data.dropna()
        box_data.append(era_data)
        box_data_labels.append(str(era))
        box_data_labels_ticks.append(i)
        i+=1
        
    # plt.subplot(rows,cols,3)
    ax3 = fig.add_subplot(gs[1,0])
        
    ax3.boxplot(box_data)
    ax3.set_title("Mean and Spread for Build Era vs Consumption (kWh)")
    ax3.set_ylabel('Daily Electricity Consumption (kWh)')
    ax3.set_xlabel("Build Era of Property")
    ax3.set_xticks(ticks=box_data_labels_ticks,labels=box_data_labels)
    plt.xticks(rotation=90)

    # plot average daily average electricity usage (kWh):
    # data in format for box plot:
    box_data = []
    box_data_labels = []
    box_data_labels_ticks = []
    property_metadata.sort_values(by=['property_occupants'], inplace=True)
    property_metadata_by_occ_split = property_metadata.groupby(property_metadata.property_occupants)
    i = 1
    for occ in property_metadata["property_occupants"].unique():
        occ_data = property_metadata_by_occ_split.get_group(occ)["average_usage_flat_kwh"]
        occ_data = occ_data.dropna()
        box_data.append(occ_data)
        box_data_labels.append(str(occ))
        box_data_labels_ticks.append(i)
        i+=1

    # plot average daily average electricity usage (kWh) vs occupants:
    # plt.subplot(rows,cols,4)
    ax4 = fig.add_subplot(gs[1,1])
        
    ax4.boxplot(box_data)
    ax4.set_title("Mean and Spread for Number of Occupants vs Consumption (kWh)")
    ax4.set_ylabel('Daily Electricity Consumption (kWh)')
    ax4.set_xlabel("Number of Occupants")
    ax4.set_xticks(ticks=box_data_labels_ticks,labels=box_data_labels)
    

    # plot average monthly usages for occupants as bars:
    ax5 = fig.add_subplot(gs[2,:])
    # plt.subplot(3,1,3)

    # montly averages for occupants:
    month_avs = []
    dl_monthly_averages_kwh.sort_values(by=['month'], inplace=True)
    monthly_averages_kwh = dl_monthly_averages_kwh.groupby(dl_monthly_averages_kwh.month)
    for m in dl_monthly_averages_kwh["month"].unique():
        month_data = monthly_averages_kwh.get_group(m)["usage_kwh"]
        month_data = month_data.dropna()
        av = np.mean(month_data)
        month_avs.append(av)

    xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    bars = plt.bar(xlabels, month_avs, label ='average monthly usage (kWh)')
    plt.bar_label(bars, fmt='{:,.0f}')
    plt.xlabel('Month of Year') 
    plt.ylabel('Monthly Electricity Consumption (kWh)') 
    plt.title('Total Monthly Electricity Consumption for Each Month (kWh)') 


    # plot average daily usages for months as bars:
    ax5 = fig.add_subplot(gs[3,:])
    # plt.subplot(3,1,3)

    # daily averages for occupants:
    month_avs = []
    dl_monthly_averages_kwh.sort_values(by=['month'], inplace=True)
    monthly_averages_kwh = dl_monthly_averages_kwh.groupby(dl_monthly_averages_kwh.month)
    for m in dl_monthly_averages_kwh["month"].unique():
        month_data = monthly_averages_kwh.get_group(m)["usage_kwh"]
        month_data = month_data.dropna()
        days = [0,31,28,31,30,31,30,31,31,30,31,30,31]
        av = np.mean(month_data)/days[m]
        month_avs.append(av)

    xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    bars = plt.bar(xlabels, month_avs, label ='average monthly usage (kWh)')
    plt.xlabel('Month of Year') 
    plt.ylabel('Daily Electricity Consumption (kWh)') 
    plt.title('Average Daily Total Electricity Consumption for Each Month (kWh)') 
    plt.bar_label(bars, fmt='{:,.2f}')


    # show plots:
    plt.tight_layout(h_pad=hpad,w_pad=wpad)
    plt.show()

def PlotPolyModelForOccBand(x_plot,x_var_name,y_var_name,xlabel,ylabel,cat,cat_data,cat_model):
    plt.scatter(cat_data[x_var_name],cat_data[y_var_name],alpha=0.1,label="all data points")
    plt.plot(x_plot, cat_model(x_plot),color="purple")
    plt.title(f"Polynomial Model for Property with {cat} Occupants")
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    ticks = np.linspace(min(cat_data[x_var_name]),max(cat_data[x_var_name]),12)
    ticks = [round(n) for n in ticks]
    # xlabels = [("01-"+str(dt.date.fromordinal(d)).split("-")[1]) for d in ticks]
    xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]

    plt.xticks(ticks=ticks,labels=xlabels,rotation=90)
    plt.legend()

def PlotModel(dl_models_data, dl_fitted_model):
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot()

    f = "%d-%M-%Y"
    start_time = pd.to_datetime("01-01-2000",format=f).toordinal()
    end_time = pd.to_datetime("01-01-2001",format=f).toordinal()

    # occs = np.linspace(0,5,365).T
    time = np.linspace(start_time,end_time,365).T
    occs = 3*np.ones((len(time),), dtype=int)
    X = np.column_stack((occs,time))

    print(X.shape)
    elec = dl_fitted_model.predict(X)

    ax.scatter(dl_models_data["occ_data_all"])
    ax.plot(time, elec)

    plt.show()



def PlotElecVsTempModelsForOccBands(elec_vs_time_for_occ_models,average_daily_usages_kwh_combined):
    # initialise plots:
    # cop_bands_len = len(elec_vs_time_for_occ_models["property_occupants"])
    # rows = round(cop_bands_len/2)
    # cols = 2
    plt.subplots(1, 1, figsize=(15,5))
    plots_opacity = 0.5
    hpad = 2
    wpad = 0.5
    
    # data:
    occ_range = elec_vs_time_for_occ_models["property_occupants"].unique()
    occ_range.sort()
    occ_range = occ_range[occ_range >= 1]

    # plot set 1 cop vs temp diff:
    # timeseries:
    start_date = pd.Timestamp("01-01-2000").toordinal()
    end_date = pd.Timestamp("01-01-2001").toordinal()
    duration = 52
    time_series = np.linspace(start_date,end_date,duration)
    # colors = ["purple","r","orange","g","c","b"]
    colors = ["b","c","g","orange","r","purple"]

    i = 0
    for occ in occ_range:
        elec_vs_time_for_occ_models_grouped = elec_vs_time_for_occ_models.groupby(elec_vs_time_for_occ_models.property_occupants)
        elec_vs_time_for_occ_models_filt = elec_vs_time_for_occ_models_grouped.get_group(occ)

        average_daily_usages_kwh_combined_grouped = average_daily_usages_kwh_combined.groupby(average_daily_usages_kwh_combined.occ)
        average_daily_usages_kwh_combined_filt = average_daily_usages_kwh_combined_grouped.get_group(occ)
    
        # plt.subplot(5,1,i+1)
        # plot data:
        occ_data = average_daily_usages_kwh_combined_filt
        occ_model = elec_vs_time_for_occ_models_filt["fitted_models"].values[0]
        color = colors[i]
        i += 1
        plt.scatter(occ_data["day"],occ_data["usage_kwh"],color=color,alpha=0.2)
        time_series = np.linspace(1,365,365)
        plt.plot(time_series, occ_model(time_series),color=color,label=f"Predicted Daily Average Consumption for {occ} Occupants")
        # plt.title(f"Polynomial Model for Property with {cat} Occupants")
        plt.ylabel("Average Daily Electricity Consumption (kWh)")
        plt.xlabel("Date dd-mm")
        plt.ylim(top=15,bottom=0)
        plt.xlim(left=1,right=365)
        xticks = np.linspace(1,365,12)
        xticks = [round(n) for n in xticks]
        xlabels = [("01-"+str(dt.date.fromordinal(d)).split("-")[1]) for d in xticks]
        # xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]

        plt.xticks(ticks=xticks,labels=xlabels,rotation=90)
        plt.legend(loc = 1)

    # show plots 1:
    # plt.suptitle(f"Models Fitted Using Polynomial Regression for Energy Consumed (kWh) against Temperature Difference")
    plt.tight_layout(h_pad=hpad,w_pad=wpad,pad=hpad)
    plt.show()    


def PlotDailyAveragesVsTime(property_ids_accepted,property_data_all):
    # plot 5 properties that have data for jan - dec:
    # start_date = pd.Timestamp("01-01-2000").toordinal()
    # end_date = pd.Timestamp("01-01-2001").toordinal()
    property_data_with_dates = []
    for property_data in property_data_all:
        if property_data is None:
            continue
        dates = property_data.index
        dates = [pd.to_datetime(d,format="%Y-%m-%d %h:%m:%s") for d in dates]
        timestamps = [pd.Timestamp(d) for d in dates]
        # print(timestamps[0],timestamps[(len(timestamps)-1)])
        has_start = [t for t in timestamps if (t.month==1)]
        if (len(has_start)==0):
            continue
        # print(has_start[0])
        year1 = has_start[0].year
        has_end = [t for t in timestamps if (t.month==12 and t.year==(year1))]
        if (len(has_end)==0):
            continue
        # print(has_end[-1]) 
        idx = [(t.year==year1) for t in timestamps]
        property_data = property_data[idx]
        property_data_with_dates.append(property_data)
    # property_data_to_plot = property_data_with_dates[::30]
    print(len(property_data_with_dates))
    property_data_to_plot = property_data_with_dates[::4][0:4]

    # initialise plots:
    rows = 4
    cols = 1
    plt.subplots(nrows=rows, ncols=cols, figsize=(12,4*rows))
    plt.suptitle(f"Visualisation of Daily Total Electricity Consumption (kWh) for Five Randomly Selected Homes")

    i = 0
    for property_data in property_data_to_plot:
        if(property_data is None):
            continue

        # property data:
        # property_data = property_data_all[i]
        plots_opacity = 0.7

        # plot daily electricity usage (kWh):
        plt.subplot(rows,cols,i+1)
        plt.title(f"Random Property {i+1}")
        plt.scatter(property_data.index,property_data["electricity_reading_kwh"],
                    label="electricity_consumption_kwh",alpha=plots_opacity)
        # plot UK average consumption (kWh) according to sources: https://www.ofgem.gov.uk/information-consumers/energy-advice-households/average-gas-and-electricity-use-explained
        # electricity consumption doesn't change significantly with time of year so can plot flat averages to compare:
        average_consumtion_uk_low = 5
        average_consumtion_uk_med = 7
        average_consumtion_uk_high = 10
        plt.axhline(average_consumtion_uk_low,label="average_consumtion_uk_low_usage",color="g",linestyle="--")
        plt.axhline(average_consumtion_uk_med,label="average_consumtion_uk_med_usage",color="y",linestyle="--")
        plt.axhline(average_consumtion_uk_high,label="average_consumtion_uk_high_usage",color="r",linestyle="--")
        plt.xlabel("Date yy-mm")
        plt.ylabel('Daily Electricity Consumption (kWh)')
        # start = property_data.index[0].toordinal()
        # end = property_data.index[-1].toordinal()

        xlabels = ["January","February","March","April","May","June","July","August","September","October","November","December"]
        xticks = property_data.index[::31]
        plt.xticks(ticks=xticks,labels=xlabels,rotation=90)
        plt.ylim(0,20)
        plt.legend(loc=1)
        i+=1
    
    # show plots:
    hpad = 2
    wpad = 0.5
    plt.tight_layout(h_pad=hpad,w_pad=wpad,pad=hpad)
    plt.show()

CreatePlots(dl_property_ids_accepted, dl_property_metadata, dl_elec_vs_date_models, dl_property_data_all,dl_monthly_averages_kwh,dl_average_daily_usages_kwh_combined)